In [1]:
from youtube_transcript_api import YouTubeTranscriptApi

# Function to get the transcript for a YouTube video with fallback to 'en-IN'
def get_transcript_with_fallback(video_id):
    try:
        # Attempt to fetch the transcript with default language preferences
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        return transcript
    except Exception as e:
        # Check if 'en-IN' is available
        try:
            available_transcripts = YouTubeTranscriptApi.list_transcripts(video_id)
            if 'en-IN' in [t.language_code for t in available_transcripts]:
                # Fetch the 'en-IN' transcript
                return available_transcripts.find_transcript(['en-IN']).fetch()
            else:
                # If 'en-IN' is not available, re-raise the original exception
                raise Exception(f"Error fetching transcript for video ID {video_id}: {e}")
        except Exception as inner_e:
            raise Exception(f"Error fetching transcript for video ID {video_id}: {inner_e}")

# Function to generate timestamps from a transcript
def generate_timestamps(transcript):
    timestamps = []
    if isinstance(transcript, list):  # Ensure transcript is valid
        for item in transcript:
            timestamps.append(f"{item['start']} - {item['text']}")
    else:
        timestamps.append(transcript)  # If there's an error, log the error message
    return timestamps

def extract_video_id(youtube_url):
    parsed_url = urlparse(youtube_url)
    query_params = parse_qs(parsed_url.query)
    video_id = query_params.get("v")
    return video_id[0] if video_id else None

ModuleNotFoundError: No module named 'youtube_transcript_api'

In [ ]:
for video_id in video_ids:
    try:
        transcript = get_transcript_with_fallback(video_id)
        timestamps = generate_timestamps(transcript)
        timestamps_data.append({"Video ID": video_id, "Timestamps": "\n".join(timestamps)})
    except Exception as e:
        timestamps_data.append({"Video ID": video_id, "Timestamps": str(e)})

# Save the timestamps to a DataFrame and then to an Excel file
timestamps_df = pd.DataFrame(timestamps_data)

# Save to Excel
output_file = '/Users/mandy/Desktop/NLP/Project/youtube_video_timestamps_with_fallback.xlsx'
timestamps_df.to_excel(output_file, index=False)